# Import package

In [1]:
import time
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
pd.set_option("max_colwidth", 200)

from pandarallel import pandarallel
pandarallel.initialize()


# from pandarallel import pandarallel
# num_workers =  72
# pandarallel.initialize(nb_workers = num_workers)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Get all the urls ( skip )

In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.chrome.options import Options
from icecream import ic
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
import os

chrome_options = Options()
# chrome_options.headless = True
chrome_drive = './chromedriver'
os.environ['webdrive.chrome.drive'] = chrome_drive

In [5]:
url =  'https://phr.org/our-work/resources/?_sf_ppp=100'
drive = webdriver.Chrome(options=chrome_options, executable_path=chrome_drive)
drive.get(url)

In [15]:
while True:
    next_find = drive.find_element_by_xpath("//*[contains(text(), 'Load More')]")
    next_find.click()
    time.sleep(2)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=88.0.4324.150)


In [17]:
soup = BeautifulSoup(drive.page_source, 'html.parser')
rows_find = next_soup.find_all('div', class_ = 'teaser--content')
len(rows_find)  # should be 2171

2171

In [62]:
values = []
for rows in rows_find:
    category = rows.find_all('div', class_="teaser--meta")
    category = category[0].text if len(category)>0  else ''
        
    title = rows.h3.a.text
    link = rows.h3.a['href']
    time = rows.find_all('div', class_="teaser--posted-on")[0].text

    terms = rows.find_all('div', class_="teaser--terms")
    terms = terms[0].text if len(terms)>0  else ''
    
    values.append([category, title, link, time, terms])

In [63]:
df  = pd.DataFrame(values, columns = ['category', 'title', 'link', 'time', 'terms'])

In [64]:
# df.to_csv('phr_raw.csv', index= False)

# Continue extract html per record

In [2]:
df= pd.read_csv('phr_raw.csv', header= [0])
df

,category,title,link,time,terms
0,Other,The Human Rights Implications of Global COVID-19 Vaccine Distribution,https://phr.org/our-work/resources/the-human-rights-implications-of-global-covid-19-vaccine-distribution/,"February 25, 2021","COVID-19, Global"
1,Open Letter,An Open Letter to President Biden,https://phr.org/our-work/resources/an-open-letter-to-president-biden/,"February 23, 2021","COVID-19, United States"
2,Press Release,"More Than 220 U.S. Leaders, Public Health Experts Call for a People’s Vaccine",https://phr.org/news/more-200-u-s-leaders-public-health-experts-call-for-a-peoples-vaccine/,"February 23, 2021","COVID-19, United States"
3,Press Release,"Survey of Health Workers at Well-Resourced U.S. Facilities Reveals Protective Equipment Shortages, Workplace Retaliation, Lack of Guidelines During Early Months of COVID-19 Pandemic",https://phr.org/news/survey-of-health-workers-at-well-resourced-u-s-facilities-reveals-protective-equipment-shortages-workplace-retaliation-lack-of-guidelines-during-early-months-of-covid-19-pande...,"February 23, 2021","COVID-19, United States"
4,Report,Silenced and Endangered: Clinicians’ Human Rights and Health Concerns about Their Facilities’ Response to COVID-19,https://phr.org/our-work/resources/silenced-and-endangered/,"February 23, 2021","COVID-19, United States"
...,...,...,...,...,...
2166,Report,El Salvador: Health Care Under Siege,https://phr.org/our-work/resources/el-salvador-health-care-under-siege/,"February 1, 1990","Attacks on Health Care, El Salvador"
2167,Report,Winds of Death,https://phr.org/our-work/resources/winds-of-death/,"January 1, 1989","Chemical Weapons, Iraq"
2168,Report,Medical Mission to Czechoslovakia,https://phr.org/our-work/resources/medical-mission-to-czechoslovakia/,"July 28, 1988",NaN
2169,Report,Panama 1987: Health Consequences of Police and Military Actions,https://phr.org/our-work/resources/panama-1987-health-consequences-of-police-and-military-actions/,"April 1, 1988","Attacks on Health Care, Panama"


In [6]:
def extract_text(url, recursive= False):
    coverpage = requests.get(url).content
    soup = BeautifulSoup(coverpage, 'html.parser')
    
    main_html_find = soup.find_all('div', class_="entry-content")
    main_html = main_html_find[0]
    paragraphs = main_html.find_all('p', recursive= recursive)
    if len(paragraphs):
        text = '\n'.join([p.text for p in paragraphs])
    else:
        text = ''
    return pd.Series([str(main_html), text])

In [4]:
df[['html','text']]=\
df['link'].parallel_apply(extract_text).values

In [5]:
df_no_text  = df.loc[df.text=='',:].copy()
df_no_text

,category,title,link,time,terms,html,text
349,Multimedia,"PHR’s Partnership with Dr. Denis Mukwege and Panzi Hospital, DRC",https://phr.org/our-work/resources/phrs-partnership-with-dr-denis-mukwege-and-panzi-hospital-drc/,"December 6, 2018","Sexual Violence, Democratic Republic of Congo","<div class=""entry-content"">\n<figure class=""wp-block-embed-youtube wp-block-embed is-type-video is-provider-youtube wp-has-aspect-ratio wp-embed-aspect-16-9""><div class=""wp-block-embed__wrapper"">\...",
386,Multimedia,Landmark child rape trial in DRC’s Kavumu,https://phr.org/our-work/resources/landmark-child-rape-trial-in-drcs-kavumu/,"September 6, 2018","Sexual Violence, Democratic Republic of Congo","<div class=""entry-content"">\n<div class=""phr-social-share-icons"" id=""social-share"">\n<div class=""phr-social-share-icons__inner"">\n<a aria-label=""Share Landmark child rape trial in DRC’s Kavumu on ...",
505,Blog,Parents of Congolese Child Rape Victims Testify in Landmark Trial,https://phr.org/our-work/resources/parents-of-congolese-child-rape-victims-testify-in-landmark-trial/,"December 11, 2017","Sexual Violence, Democratic Republic of Congo","<div class=""entry-content"">\n<table border=""0"" cellpadding=""5"" cellspacing=""5"" style=""width: 665px;"">\n<tbody>\n<tr>\n<td>\n<div id=""image-844678032"">\n<div class=""image"" style=""width: 600px;"">\n<...",
506,Blog,“Shot From Behind While Fleeing” – PHR’s Dr. Homer Venters Reports from the Myanmar-Bangladesh Border,https://phr.org/our-work/resources/shot-from-behind-while-fleeing-phrs-dr-homer-venters-reports-from-the-myanmar-bangladesh-border/,"December 8, 2017","Rohingya, Bangladesh","<div class=""entry-content"">\n<table cellpadding=""5"" cellspacing=""0"" style=""width: 600px; height: 100%;"">\n<tbody>\n<tr>\n<td>One Rohingya man suffered a compound fracture in his leg when he was sa...",
510,Blog,PHR Awarded 2017 Dodd Prize in Human Rights,https://phr.org/our-work/resources/phr-awarded-2017-dodd-prize-in-human-rights/,"November 28, 2017",NaN,"<div class=""entry-content"">\n<table border=""0"" cellpadding=""5"" cellspacing=""5"" style=""width: 660px;"">\n<tbody>\n<tr>\n<td>\n<div id=""convio-content-842653320"" style=""display: inline;""><!-- No rend...",
668,Report,Lethal in Disguise,https://phr.org/our-work/resources/lethal-in-disguise/,"March 1, 2016","Weapons, Global","<div class=""entry-content"">\n<div class=""wp-block-group""><div class=""wp-block-group__inner-container"">\n<p>Read fact sheets about <a href=""https://phr.org/wp-content/uploads/2020/06/PHR_INCLO_Fact...",
695,Multimedia,PHR’s Program on Sexual Violence in Conflict Zones,https://phr.org/our-work/resources/phrs-program-on-sexual-violence-in-conflict-zones-2/,"December 1, 2015","Sexual Violence, Democratic Republic of Congo","<div class=""entry-content"">\n<div class=""phr-social-share-icons"" id=""social-share"">\n<div class=""phr-social-share-icons__inner"">\n<a aria-label=""Share PHR’s Program on Sexual Violence in Conflict ...",
975,Blog,PHR's Letter to the Turkish Ministry of Health,https://phr.org/our-work/resources/phrs-letter-to-the-turkish-ministry-of-health/,"July 1, 2013","Persecution of Health Workers, Turkey","<div class=""entry-content"">\n<div>\n<div class=""WordSection1"">\n<p><em><strong>PHR sent this letter to the Turkish Ministry of Health expressing our deep concern over the ministry's efforts to pas...",
984,Blog,Burma Must Take Steps to Quell Ethnic Violence,https://phr.org/our-work/resources/burma-must-take-steps-to-quell-ethnic-violence/,"May 30, 2013","Mass Atrocities, Burma","<div class=""entry-content"">\n<div class=""content-styles"">\n<p><em>This post originally appeared on <a href=""http://www.dvb.no/analysis/burma-must-take-steps-to-quell-ethnic-violence/28541"" target=...",
1663,Other,Letter to Iranian Authorities Concerning the Suspicious Death of Dr. Ramin Pourandarjani,https://phr.org/our-work/resources/letter-to-iranian-authorities-concerning-the-suspicious-death-of-dr-r

In [6]:
df_no_text.shape

(22, 7)

Further handle those rows that don't have text. Find text recursively

In [7]:
df_no_text[['html','text']] =  df_no_text['link'].parallel_apply(extract_text, recursive= True).values
df_no_text

,category,title,link,time,terms,html,text
349,Multimedia,"PHR’s Partnership with Dr. Denis Mukwege and Panzi Hospital, DRC",https://phr.org/our-work/resources/phrs-partnership-with-dr-denis-mukwege-and-panzi-hospital-drc/,"December 6, 2018","Sexual Violence, Democratic Republic of Congo","<div class=""entry-content"">\n<figure class=""wp-block-embed-youtube wp-block-embed is-type-video is-provider-youtube wp-has-aspect-ratio wp-embed-aspect-16-9""><div class=""wp-block-embed__wrapper"">\...",
386,Multimedia,Landmark child rape trial in DRC’s Kavumu,https://phr.org/our-work/resources/landmark-child-rape-trial-in-drcs-kavumu/,"September 6, 2018","Sexual Violence, Democratic Republic of Congo","<div class=""entry-content"">\n<div class=""phr-social-share-icons"" id=""social-share"">\n<div class=""phr-social-share-icons__inner"">\n<a aria-label=""Share Landmark child rape trial in DRC’s Kavumu on ...",
505,Blog,Parents of Congolese Child Rape Victims Testify in Landmark Trial,https://phr.org/our-work/resources/parents-of-congolese-child-rape-victims-testify-in-landmark-trial/,"December 11, 2017","Sexual Violence, Democratic Republic of Congo","<div class=""entry-content"">\n<table border=""0"" cellpadding=""5"" cellspacing=""5"" style=""width: 665px;"">\n<tbody>\n<tr>\n<td>\n<div id=""image-844678032"">\n<div class=""image"" style=""width: 600px;"">\n<...",The parents of a group of little Congolese girls who were savagely raped during a three-year reign of terror in eastern Democratic Republic of the Congo (DRC) took the stand today in a landmark tr...
506,Blog,“Shot From Behind While Fleeing” – PHR’s Dr. Homer Venters Reports from the Myanmar-Bangladesh Border,https://phr.org/our-work/resources/shot-from-behind-while-fleeing-phrs-dr-homer-venters-reports-from-the-myanmar-bangladesh-border/,"December 8, 2017","Rohingya, Bangladesh","<div class=""entry-content"">\n<table cellpadding=""5"" cellspacing=""0"" style=""width: 600px; height: 100%;"">\n<tbody>\n<tr>\n<td>One Rohingya man suffered a compound fracture in his leg when he was sa...","Stories like these, and of mass rapes, mass killings, and mass graves, are some of the accounts being heard by a team of doctors from Physicians for Human Rights that is currently on the Myanmar-B..."
510,Blog,PHR Awarded 2017 Dodd Prize in Human Rights,https://phr.org/our-work/resources/phr-awarded-2017-dodd-prize-in-human-rights/,"November 28, 2017",NaN,"<div class=""entry-content"">\n<table border=""0"" cellpadding=""5"" cellspacing=""5"" style=""width: 660px;"">\n<tbody>\n<tr>\n<td>\n<div id=""convio-content-842653320"" style=""display: inline;""><!-- No rend...","McKay then delivered the following keynote remarks to the audience: \nAs Senator Dodd said, I’m Donna McKay, and I’m the executive director of Physicians for Human Rights.\nOn behalf of our found..."
668,Report,Lethal in Disguise,https://phr.org/our-work/resources/lethal-in-disguise/,"March 1, 2016","Weapons, Global","<div class=""entry-content"">\n<div class=""wp-block-group""><div class=""wp-block-group__inner-container"">\n<p>Read fact sheets about <a href=""https://phr.org/wp-content/uploads/2020/06/PHR_INCLO_Fact...","Read fact sheets about acoustic weapons, directed energy devices, rubber bullets (and other kinetic impact projectiles), stun grenades (and other disorientation devices), tear gas (and other chemi..."
695,Multimedia,PHR’s Program on Sexual Violence in Conflict Zones,https://phr.org/our-work/resources/phrs-program-on-sexual-violence-in-conflict-zones-2/,"December 1, 2015","Sexual Violence, Democratic Republic of Congo","<div class=""entry-content"">\n<div class=""phr-social-share-icons"" id=""social-share"">\n<div class=""phr-social-share-icons__inner"">\n<a aria-label=""Share PHR’s Program on Sexual Violence in Conflict ...",
975,Blog,PHR's Letter to the Turkish Ministry of Health,https://phr.org/our-work/resources/phrs-letter-to-the-turkish-ministry-of-health/,"July 1, 2013","Persecution of Health Workers, Turkey

In [8]:
df.loc[df.text=='',:] = df_no_text.values
df.loc[df.text=='',:].shape

(15, 7)

In [19]:
df.text.str.len().describe()

count      2156.000000
mean       3866.494434
std        9772.875373
min           1.000000
25%        1477.750000
50%        2628.500000
75%        4104.250000
max      179545.000000
Name: text, dtype: float64

In [20]:
df = df[df.text.str.len()>150]

In [22]:
# df.to_csv('phr_text.csv', index= False)

# Further process

In [2]:
df = pd.read_csv('phr_text.csv', header= [0])

In [3]:
df1 = df.copy()
df1

,category,title,link,time,terms,html,text
0,Other,The Human Rights Implications of Global COVID-19 Vaccine Distribution,https://phr.org/our-work/resources/the-human-rights-implications-of-global-covid-19-vaccine-distribution/,"February 25, 2021","COVID-19, Global","<div class=""entry-content"">\n<div class=""wp-block-file aligncenter""><a class=""wp-block-file__button"" download="""" href=""https://phr.org/wp-content/uploads/2021/02/The-Human-RIghts-Implications-of-G...","The novel coronavirus has spread rapidly around the globe, infecting more than 100 million people and killing more than two million, with the toll rising daily. There are more than 25 million case..."
1,Open Letter,An Open Letter to President Biden,https://phr.org/our-work/resources/an-open-letter-to-president-biden/,"February 23, 2021","COVID-19, United States","<div class=""entry-content"">\n<p><em>This public letter to President Biden was updated on February 23, 2021 with new signers, totaling 222.</em></p>\n<p>A safe, effective and accessible COVID-19 v...","This public letter to President Biden was updated on February 23, 2021 with new signers, totaling 222.\nA safe, effective and accessible COVID-19 vaccine is vital to bring an end to today’s dual ..."
2,Press Release,"More Than 220 U.S. Leaders, Public Health Experts Call for a People’s Vaccine",https://phr.org/news/more-200-u-s-leaders-public-health-experts-call-for-a-peoples-vaccine/,"February 23, 2021","COVID-19, United States","<div class=""entry-content"">\n<p>More than 220 leaders from the fields of public health, medicine, global development, and racial justice joined faith leaders, economists, Nobel laureates, former m...","More than 220 leaders from the fields of public health, medicine, global development, and racial justice joined faith leaders, economists, Nobel laureates, former members of Congress and artists t..."
3,Press Release,"Survey of Health Workers at Well-Resourced U.S. Facilities Reveals Protective Equipment Shortages, Workplace Retaliation, Lack of Guidelines During Early Months of COVID-19 Pandemic",https://phr.org/news/survey-of-health-workers-at-well-resourced-u-s-facilities-reveals-protective-equipment-shortages-workplace-retaliation-lack-of-guidelines-during-early-months-of-covid-19-pande...,"February 23, 2021","COVID-19, United States","<div class=""entry-content"">\n<p>A survey of 901 health workers, largely people who worked at well-resourced health facilities in the United States, about their experiences during the early months ...","A survey of 901 health workers, largely people who worked at well-resourced health facilities in the United States, about their experiences during the early months of the COVID-19 emergency highli..."
4,Report,Silenced and Endangered: Clinicians’ Human Rights and Health Concerns about Their Facilities’ Response to COVID-19,https://phr.org/our-work/resources/silenced-and-endangered/,"February 23, 2021","COVID-19, United States","<div class=""entry-content"">\n<a data-scroll="""" id=""phr_toc_0"" name=""phr_toc_0""></a><h2>Executive Summary</h2>\n<p>Health care workers have been fired, faced new restrictions, and even been detaine...","Health care workers have been fired, faced new restrictions, and even been detained for voicing their safety concerns. Many governments and employers have praised health care workers as heroes, wh..."
...,...,...,...,...,...,...,...
2135,Report,El Salvador: Health Care Under Siege,https://phr.org/our-work/resources/el-salvador-health-care-under-siege/,"February 1, 1990","Attacks on Health Care, El Salvador","<div class=""entry-content"">\n<p>In June 1989, PHR sent a medical mission to El Salvador to investigate and report on allegations of violations of medical neutrality and other human rights abuses c...","In June 1989, PHR sent a medical mission to El Salvador to investigate and report on allegations of violations of medical neutrality and other human rights abuses committed by both sides in the ci..."
2136

In [6]:
df1[df1.text.str.len()<180]

,category,title,link,time,terms,html,text
384,Multimedia,PHR honors Iraqi Dr. Nagham Hasan,https://phr.org/our-work/resources/phr-honors-iraqi-dr-nagham-hasan/,"September 6, 2018","Sexual Violence, Iraq","<div class=""entry-content"">\n<p>PHR salutes Dr. Hasan for her extraordinary work treating and supporting survivors of sexual slavery and genocide perpetrated by ISIS against Iraq’s Yazidi minority...",PHR salutes Dr. Hasan for her extraordinary work treating and supporting survivors of sexual slavery and genocide perpetrated by ISIS against Iraq’s Yazidi minority.
1807,Multimedia,Tortured Logic,https://phr.org/our-work/resources/tortured-logic/,"April 21, 2009","Torture, Afghanistan","<div class=""entry-content"">\n<p>How Medical Professionals Rationalized the Bush Administration's Torture Program.</p>\n<div class=""player""><iframe allowfullscreen="""" frameborder=""0"" height=""280"" s...",How Medical Professionals Rationalized the Bush Administration's Torture Program.\nPHR responds to the release of the Senate Armed Forces Committee report on detainee abuse.
1920,Report,Forensic Documentation of Torture and Ill Treatment in Mexico,https://phr.org/our-work/resources/forensic-documentation-of-torture-and-ill-treatment-in-mexico/,"December 5, 2008","Torture, Mexico","<div class=""entry-content"">\n<p>PHR documents the Mexican government’s historic attempt and ultimate failure to implement international standards of forensic evaluations of torture and ill treatme...",PHR documents the Mexican government’s historic attempt and ultimate failure to implement international standards of forensic evaluations of torture and ill treatment.
1985,Report,UNAMID Deployment on the Brink,https://phr.org/our-work/resources/unamid-deployment-on-the-brink/,"December 20, 2007","Mass Atrocities, Sudan","<div class=""entry-content"">\n<p>The hybrid peacekeeping force for Darfur is being set up to fail, a group of 35 nongovernmental organizations warned in a report released December 20, 2007.</p>\n<d...","The hybrid peacekeeping force for Darfur is being set up to fail, a group of 35 nongovernmental organizations warned in a report released December 20, 2007."


In [8]:
df_sample = df1.sample(n=10).reset_index(drop=True)
for i,j in zip(df_sample.link, df_sample.text):
    print(i)
    print(j)
    print('\n')

https://phr.org/our-work/resources/special-rapporteur-mendez-assessing-torture-prohibition-measures-in-tajikistan/
United Nations Special Rapporteur on torture and other cruel, inhuman, or degrading treatment or punishment, Juan Méndez, is currently visiting Tajikistan to assess measures taken by the Tajik government to bring its torture prohibition legislation into compliance with international standards.
Mr. Mendez has noted that while the Tajik government has taken some steps to reduce the incidence of human rights violations, “there remain significant gaps in legislation, policies and law enforcement practices…” He specifically noted his intention to examine “…how the authorities comply with the universal obligations to afford victims of torture an effective remedy and to investigate, prosecute, and punish every episode of torture.”
This is Mr. Mendez’s second official visit to Central Asia since he was appointed Special Rapporteur in November 2010. Following his December 2011 visi

In [ ]:
df1 = df1[['category','title','link',		'text']]

In [5]:
# df = pd.read_csv('phr.csv', header= [0])
# df= df.rename(columns={'link':'url'})
df

,category,title,url,date,tags,text
0,Other,The Human Rights Implications of Global COVID-19 Vaccine Distribution,https://phr.org/our-work/resources/the-human-rights-implications-of-global-covid-19-vaccine-distribution/,"February 25, 2021","COVID-19, Global","The novel coronavirus has spread rapidly around the globe, infecting more than 100 million people and killing more than two million, with the toll rising daily. There are more than 25 million case..."
1,Open Letter,An Open Letter to President Biden,https://phr.org/our-work/resources/an-open-letter-to-president-biden/,"February 23, 2021","COVID-19, United States","This public letter to President Biden was updated on February 23, 2021 with new signers, totaling 222.\nA safe, effective and accessible COVID-19 vaccine is vital to bring an end to today’s dual ..."
2,Press Release,"More Than 220 U.S. Leaders, Public Health Experts Call for a People’s Vaccine",https://phr.org/news/more-200-u-s-leaders-public-health-experts-call-for-a-peoples-vaccine/,"February 23, 2021","COVID-19, United States","More than 220 leaders from the fields of public health, medicine, global development, and racial justice joined faith leaders, economists, Nobel laureates, former members of Congress and artists t..."
3,Press Release,"Survey of Health Workers at Well-Resourced U.S. Facilities Reveals Protective Equipment Shortages, Workplace Retaliation, Lack of Guidelines During Early Months of COVID-19 Pandemic",https://phr.org/news/survey-of-health-workers-at-well-resourced-u-s-facilities-reveals-protective-equipment-shortages-workplace-retaliation-lack-of-guidelines-during-early-months-of-covid-19-pande...,"February 23, 2021","COVID-19, United States","A survey of 901 health workers, largely people who worked at well-resourced health facilities in the United States, about their experiences during the early months of the COVID-19 emergency highli..."
4,Report,Silenced and Endangered: Clinicians’ Human Rights and Health Concerns about Their Facilities’ Response to COVID-19,https://phr.org/our-work/resources/silenced-and-endangered/,"February 23, 2021","COVID-19, United States","Health care workers have been fired, faced new restrictions, and even been detained for voicing their safety concerns. Many governments and employers have praised health care workers as heroes, wh..."
...,...,...,...,...,...,...
2135,Report,El Salvador: Health Care Under Siege,https://phr.org/our-work/resources/el-salvador-health-care-under-siege/,"February 1, 1990","Attacks on Health Care, El Salvador","In June 1989, PHR sent a medical mission to El Salvador to investigate and report on allegations of violations of medical neutrality and other human rights abuses committed by both sides in the ci..."
2136,Report,Winds of Death,https://phr.org/our-work/resources/winds-of-death/,"January 1, 1989","Chemical Weapons, Iraq","In the late 1980’s Iraq’s army attacked Kurdish villages with poison gas. PHR documented the injuries and deaths of Kurdish civilians from these attacks, finding that Iraqi aircraft used bombs con..."
2137,Report,Medical Mission to Czechoslovakia,https://phr.org/our-work/resources/medical-mission-to-czechoslovakia/,"July 28, 1988",NaN,"PHR is the first human rights organization to be allowed into Czechoslovakia to examine political prisoners. This joint report with Helsinki Watch, entitled “Medical Mission to Czechoslovakia” cal..."
2138,Report,Panama 1987: Health Consequences of Police and Military Actions,https://phr.org/our-work/resources/panama-1987-health-consequences-of-police-and-military-actions/,"April 1, 1988","Attacks on Health Care, Panama",The goal of this fact-finding mission was to investigate and document widespread violations of human rights and medical neutrality which were alleged to have occurred in Panama following civilian ...


In [8]:
df[df.text.duplicated()]

,category,title,url,date,tags,text
402,Press Release,Letter from former UN Special Rapporteur on Torture Juan Mendez urging APA to maintain ban on military psychologists at Guantanamo,https://phr.org/news/letter-from-former-un-special/,"August 6, 2018",Torture,"For Immediate Release\nRead the full letter here >>\nFormer UN Special Rapporteur on Torture and Other Cruel, Inhuman or Degrading Treatment or Punishment Juan E. Méndez has issued a letter to the..."
413,Press Release,“Please Tell the World What They Have Done to Us”,https://phr.org/news/chut-pyin-report/,"July 20, 2018",NaN,For Immediate Release\nA body of forensic medical evidence released today clearly indicates that Rohingya Muslims suffered grave human rights abuses at the hands of Myanmar security forces and Rak...
416,Press Release,PHR Echoes Concerns Raised by Homeland Security Health Experts about Family Detention,https://phr.org/news/family-detention-medical-concerns/,"July 19, 2018",Asylum,For Immediate Release\nPhysicians for Human Rights (PHR) echoes today the serious concerns expressed by two Department of Homeland Security (DHS) medical experts who have spoken out about the inhu...
418,Press Release,Survivors of International Crimes Still Await Justice on 20th Anniversary of Rome Statute,https://phr.org/news/rome-statute-20-anniversary/,"July 17, 2018",NaN,For Immediate Release\nPhysicians for Human Rights (PHR) said today that the promise of justice for international crimes remains elusive 20 years after the adoption of the treaty that established ...
420,Press Release,"As Fighting Intensifies, Daraa Civilians and Hospitals Must be Spared",https://phr.org/news/daraa-hospital-attacks/,"July 2, 2018",NaN,"For Immediate Release\nPhysicians for Human Rights (PHR) is deeply concerned by the news of yet more indiscriminate attacks on health care sites in Syria. In less than a week, PHR has received cre..."
422,Press Release,Physicians for Human Rights Condemns U.S. Attempt to Eliminate Legal Protections for Immigrant Children,https://phr.org/news/legal-protections-for-immigrant-children/,"June 21, 2018",Asylum,For Immediate Release\nPhysicians for Human Rights (PHR) condemned today’s request by the U.S. Department of Justice (DOJ) to modify the 1997 Flores settlement agreement by seeking exemption from ...
424,Press Release,PHR Denounces Executive Order to Replace Separation of Children with Family Detention,https://phr.org/news/phr-denounces-executive-order/,"June 20, 2018",Asylum,For Immediate Release\nPresident Trump has signed an Executive Order to end his own administration’s policy of separating migrant and asylum-seeking children from their parents in the face of a ma...
427,Press Release,U.S. Withdrawal from Human Rights Council Undermines Accountability,https://phr.org/news/us-withdrawal-from-human/,"June 19, 2018",NaN,"For Immediate Release\nPhysicians for Human Rights (PHR) is deeply disappointed and concerned by the United States government’s decision to withdraw from the UN Human Rights Council, a move it cal..."
429,Press Release,Families Must be Together in Community-Based Settings: Tent Cities are Inhumane.,https://phr.org/news/families-must-be-together-in-community-based-settings-tent-cities-are-inhumane/,"June 14, 2018",NaN,"For Immediate Release\nPhysicians for Human Rights (PHR) welcomes recent steps by Congress to end the Trump administration’s policy of separating families at the U.S.-Mexico border, but rejects an..."
433,Press Release,Civilians Must Be Protected as Hudeidah Comes Under Assault,https://phr.org/news/civilians-must-be-protected-hudeidah/,"June 13, 2018",NaN,"For Immediate Release\nPhysicians for Human Rights (PHR) is horrified by an attack launched today by the Saudi-led coalition on the Yemeni city of Hudeidah, despite repeated calls for all parties ..."


In [11]:
df.text[420]

'For Immediate Release\nPhysicians for Human Rights (PHR) is deeply concerned by the news of yet more indiscriminate attacks on health care sites in Syria. In less than a week, PHR has received credible reports of large-scale airstrikes on at least six hospitals in Daraa in the south of the country, including al-Giza Hospital’s emergency department, which was completely destroyed after being targeted six times.\nHomer Venters, MD, PHR’s director of programs, said the attacks are continuing with impunity, with catastrophic consequences. We have seen unrelenting bloodshed and a pattern of targeted, deliberate, and cruel attacks on civilians, medical personnel, hospitals, and other health facilities since the start of this conflict. This is taking place against a backdrop of multiple failed attempts by the international community to impose a ceasefire in Syria. As a result, civilians are prevented from getting the medical assistance which they so sorely need,” Venters said.\n“PHR has docu

In [10]:
df.text.str.len().describe()

count      2140.000000
mean       3894.835047
std        9803.836865
min         156.000000
25%        1502.750000
50%        2653.500000
75%        4125.250000
max      179545.000000
Name: text, dtype: float64

**You can play with  the function html2text  to improve the text.  I only use \<p\> to define plain text. But there might be useless information in the tails of the stories.**

In [8]:
def html2text(html, recursive= True):
    
    main_html = BeautifulSoup(html, 'html.parser')
    
    paragraphs = main_html.find_all('p', recursive= recursive)
    if len(paragraphs):
        text = '\n'.join([p.text for p in paragraphs])
    else:
        text = ''
    return text

In [7]:
url = df1.link[100]
url

'https://phr.org/our-work/resources/with-federal-government-flailing-governors-must-protect-the-health-workers-who-protect-us/'

In [8]:
html =  df1.html[100]
html

'<div class="entry-content">\n<p><a href="https://www.bloomberg.com/opinion/articles/2020-06-26/states-must-protect-health-workers-amid-the-coronavirus-crisis"><em>Originally published by Bloomberg Opinion </em></a></p>\n<p>The federal government’s response to Covid-19 has been haphazard, mismanaged and ultimately deadly. Yet the Trump administration is trumpeting the country’s “success” against the pandemic, with the vice president recently <a href="https://www.wsj.com/articles/there-isnt-a-coronavirus-second-wave-11592327890?mod=hp_opin_pos_1">declaring</a> that the U.S. response to Covid-19 is “cause for celebration.”</p>\n<p>As doctors, we are not celebrating. With more than 125,000 people in the United States <a href="https://coronavirus.jhu.edu/map.html">dead from Covid-19</a> and new cases climbing in 29 states, this is no time to let down our guard. And while the issue may have faded from the headlines, health workers in many parts of the country <a href="https://www.medicaldev

In [10]:
print(html2text(html, recursive= True))

Originally published by Bloomberg Opinion 
The federal government’s response to Covid-19 has been haphazard, mismanaged and ultimately deadly. Yet the Trump administration is trumpeting the country’s “success” against the pandemic, with the vice president recently declaring that the U.S. response to Covid-19 is “cause for celebration.”
As doctors, we are not celebrating. With more than 125,000 people in the United States dead from Covid-19 and new cases climbing in 29 states, this is no time to let down our guard. And while the issue may have faded from the headlines, health workers in many parts of the country still lack personal protective equipment (PPE). Doctors’ and nurses’ pleas for masks and gloves may no longer be trending on social media, but the nationwide PPE crisis persists.
We have seen colleagues and friends forced to make impossible decisions — putting their own lives at risk to care for Covid-19 patients because they didn’t have the proper protective equipment. Some hea

In [12]:
print(html2text(html, recursive= False))

In [9]:
new_text = df1['html'].parallel_apply(extract_text).values

InvalidSchema: No connection adapters were found for '<div class="entry-content">\n<div class="resources--report-download-multiple">\n<strong>Downloads</strong>\n<ul>\n<li>\n<a class="report--download" href="https://phr.org/wp-content/uploads/2014/11/Burma-Thilawa-English-Report-Nov2014.pdf" id="a-foreseeable-disaster-in-burma" target="_blank" title="A Foreseeable Disaster in Burma">\n\t\t\t\t\t\t\t\tA Foreseeable Disaster in Burma\t\t\t\t\t\t\t</a>\n</li>\n<li>\n<a class="report--download" href="https://phr.org/wp-content/uploads/2014/11/burma-displacement-report-japanese-2014.pdf" id="-" target="_blank" title=" ">\n\t\t\t\t\t\t\t\tビルマにおける予期できた災害: ティラワ経済特別区の強制移転\t\t\t\t\t\t\t</a>\n</li>\n<li>\n<a class="report--download" href="https://phr.org/wp-content/uploads/2014/11/Burma-Thilawa-Executive-Summary-Nov2014.pdf" id="a-foreseeable-disaster-in-burma-executive-summary" target="_blank" title="A Foreseeable Disaster in Burma Executive Summary">\n\t\t\t\t\t\t\t\tA Foreseeable Disaster in B

In [ ]:
df[['new_text']]= new_text
df.to_csv('phr_new_text.csv', index=False)

In [11]:
df1

,category,title,link,time,terms,html,text
0,Other,The Human Rights Implications of Global COVID-19 Vaccine Distribution,https://phr.org/our-work/resources/the-human-rights-implications-of-global-covid-19-vaccine-distribution/,"February 25, 2021","COVID-19, Global","<div class=""entry-content"">\n<div class=""wp-block-file aligncenter""><a class=""wp-block-file__button"" download="""" href=""https://phr.org/wp-content/uploads/2021/02/The-Human-RIghts-Implications-of-G...","The novel coronavirus has spread rapidly around the globe, infecting more than 100 million people and killing more than two million, with the toll rising daily. There are more than 25 million case..."
1,Open Letter,An Open Letter to President Biden,https://phr.org/our-work/resources/an-open-letter-to-president-biden/,"February 23, 2021","COVID-19, United States","<div class=""entry-content"">\n<p><em>This public letter to President Biden was updated on February 23, 2021 with new signers, totaling 222.</em></p>\n<p>A safe, effective and accessible COVID-19 v...","This public letter to President Biden was updated on February 23, 2021 with new signers, totaling 222.\nA safe, effective and accessible COVID-19 vaccine is vital to bring an end to today’s dual ..."
2,Press Release,"More Than 220 U.S. Leaders, Public Health Experts Call for a People’s Vaccine",https://phr.org/news/more-200-u-s-leaders-public-health-experts-call-for-a-peoples-vaccine/,"February 23, 2021","COVID-19, United States","<div class=""entry-content"">\n<p>More than 220 leaders from the fields of public health, medicine, global development, and racial justice joined faith leaders, economists, Nobel laureates, former m...","More than 220 leaders from the fields of public health, medicine, global development, and racial justice joined faith leaders, economists, Nobel laureates, former members of Congress and artists t..."
3,Press Release,"Survey of Health Workers at Well-Resourced U.S. Facilities Reveals Protective Equipment Shortages, Workplace Retaliation, Lack of Guidelines During Early Months of COVID-19 Pandemic",https://phr.org/news/survey-of-health-workers-at-well-resourced-u-s-facilities-reveals-protective-equipment-shortages-workplace-retaliation-lack-of-guidelines-during-early-months-of-covid-19-pande...,"February 23, 2021","COVID-19, United States","<div class=""entry-content"">\n<p>A survey of 901 health workers, largely people who worked at well-resourced health facilities in the United States, about their experiences during the early months ...","A survey of 901 health workers, largely people who worked at well-resourced health facilities in the United States, about their experiences during the early months of the COVID-19 emergency highli..."
4,Report,Silenced and Endangered: Clinicians’ Human Rights and Health Concerns about Their Facilities’ Response to COVID-19,https://phr.org/our-work/resources/silenced-and-endangered/,"February 23, 2021","COVID-19, United States","<div class=""entry-content"">\n<a data-scroll="""" id=""phr_toc_0"" name=""phr_toc_0""></a><h2>Executive Summary</h2>\n<p>Health care workers have been fired, faced new restrictions, and even been detaine...","Health care workers have been fired, faced new restrictions, and even been detained for voicing their safety concerns. Many governments and employers have praised health care workers as heroes, wh..."
...,...,...,...,...,...,...,...
2135,Report,El Salvador: Health Care Under Siege,https://phr.org/our-work/resources/el-salvador-health-care-under-siege/,"February 1, 1990","Attacks on Health Care, El Salvador","<div class=""entry-content"">\n<p>In June 1989, PHR sent a medical mission to El Salvador to investigate and report on allegations of violations of medical neutrality and other human rights abuses c...","In June 1989, PHR sent a medical mission to El Salvador to investigate and report on allegations of violations of medical neutrality and other human rights abuses committed by both sides in the ci..."
2136